In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import re

In [2]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [3]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [4]:
%%capture pwd
!pwd

In [5]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('what-are-the-implications-of-decarbonisation-for-inequality', 300, 500)

# Fig 1

In [8]:
df = pd.read_excel(
    "raw/Econ_observatory_figures_(2).xlsx", sheet_name="Sheet2", usecols="A:H"
)
df["Unnamed: 0"] = df["Unnamed: 0"].ffill()
df.columns = ["cat", "subcat", "neg", "o", "pos", "hi", "hi_dis", "total"]
df = df.set_index(["cat", "subcat"])[["neg", "o", "pos", "total"]].stack().reset_index()
df.columns = ["cat", "subcat", "a", "value"]

In [18]:
f = "fig1_outcomes"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,cat,subcat,a,value
0,Building codes,Environmental effectiveness,neg,0.00
1,Building codes,Environmental effectiveness,o,0.15
2,Building codes,Environmental effectiveness,pos,0.85
3,Building codes,Environmental effectiveness,total,20.00
4,Building codes,Technological effectiveness,total,0.00


In [19]:
base = (
    alt.Chart(f1)
    .encode(
        x=alt.X(
            "cat:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                labelAlign="left",
                title="",
                titleY=-15,
                titleX=207,
                labelFontSize=11,
                labelPadding=-10,
                labelColor=scale_lightness(colors["eco-gray"], 2),
                labelFontWeight='bold',
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=-90,
                zindex=10,
                labelLimit=1000,
            ),
        )
    )
    .transform_filter('datum.subcat=="Distributional outcomes"')
)
bars = (
    base.mark_bar(opacity=0.8)
    .encode(
        y=alt.Y(
            "value:Q",
            stack=True,
            sort=["neg", "m", "pos"],
            axis=alt.Axis(
                grid=False,
                title="% of total evaluations",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                format=".0%",
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        order=alt.Order("a:N", sort="ascending"),
        color=alt.Color(
            "a:N",
            legend=None,
            scale=alt.Scale(
                range=[colors["eco-dot"], colors["eco-gray"], colors["eco-light-blue"]]
            ),
        ),
    )
    .transform_filter('datum.a!="total"')
)
line = (
    base.mark_line(color=colors["eco-turquiose"])
    .encode(
        y=alt.Y(
            "value:Q",
            stack=True,
            sort=["neg", "m", "pos"],
            axis=alt.Axis(
                grid=True,
                gridColor=colors["eco-gray"],
                gridOpacity=0.1,
                title="Number of total evaluations",
                titleX=5,
                titleY=13,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        x=alt.X(
            "cat:N",
            axis=alt.Axis(
                grid=True,
                title="",
                labelFontSize=0,
                gridColor=colors["eco-gray"],
                gridOpacity=0.1,
                tickColor=scale_lightness(colors["eco-gray"], 10),
                domain=False,
                orient="top",
            ),
        ),
    )
    .transform_filter('datum.a=="total"')
)
points = line.mark_point(color=colors["eco-turquiose"], fill=colors["eco-turquiose"])
title = alt.TitleParams(
    "Percentage of impacts on distributional outcomes by policy instrument type",
    subtitle=["positive impacts (blue), no impact (grey), negative impacts (red)"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
labels1 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": " ", "y": 0.1, "t": "negative"},
            ]
        )
    )
    .mark_text(angle=270, color=colors["eco-dot"])
    .encode(
        x=alt.X("x:N", sort=[]),
        y=alt.Y("y:Q", sort=[], stack=False),
        text="t:N",
    )
)
labels2 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": " ", "y": 0.3, "t": "neutral"},
            ]
        )
    )
    .mark_text(angle=270, color=colors["eco-gray"])
    .encode(
        x=alt.X("x:N", sort=[]),
        y=alt.Y("y:Q", sort=[], stack=False),
        text="t:N",
    )
)
labels3 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": " ", "y": 0.5, "t": "positive"},
            ]
        )
    )
    .mark_text(angle=270, color=colors["eco-light-blue"])
    .encode(
        x=alt.X("x:N", sort=[]),
        y=alt.Y("y:Q", sort=[], stack=False),
        text="t:N",
    )
)
labels0 = (
    alt.Chart(
        pd.DataFrame(
            [
                {"x": " ", "y": 0.5, "t": ""},
            ]
        )
    )
    .mark_text(angle=270)
    .encode(
        x=alt.X("x:N", sort=[]),
        y=alt.Y("y:Q", sort=[], stack=False),
        text="t:N",
    )
)
layer1 = (
    alt.vconcat(
        (bars + labels1 + labels2 + labels3).properties(height=250, width=400),
        (line + points + labels0).properties(height=80, width=400),
        spacing=5,
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png",scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "w").write(readme)
layer1

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.


alt.VConcatChart(...)

In [20]:
theme = "_dark"
line.encoding.x.axis.tickColor=colors["eco-background"]
bars.encoding.x.axis.labelColor=scale_lightness(colors["eco-gray"], 1.6)

layer1 = (
    alt.vconcat(
        (bars + labels1 + labels2 + labels3).properties(height=250, width=400),
        (line + points + labels0).properties(height=80, width=400),
        spacing=5,
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1 = layer1.configure_axisYQuantitative(labelFontSize=12)
layer1 = layer1.configure_axisXQuantitative(labelFontSize=12)
layer1.config.font="Georgia"
layer1.config.background=colors["eco-background"]
layer1.config.view.stroke=None
layer1.title.fontSize = 14
layer1.title.subtitleFontSize = 12
layer1.title.dy -= 2
layer1.title.color = colors["eco-dot"]
layer1.title.subtitleColor = colors["eco-dot"]
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png",scale_factor=2.0)
layer1.save("visualisation/" + f + theme + ".svg")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.


alt.VConcatChart(...)

# Fig 2

In [15]:
# polar chart
df = pd.read_excel(
    "raw/Econ_observatory_figures_(2).xlsx", sheet_name="Sheet2", usecols="A:H"
)
df["Unnamed: 0"] = df["Unnamed: 0"].ffill()
df.columns = ["cat", "subcat", "neg", "o", "pos", "hi", "hi_dis", "total"]
df = df.set_index(["cat", "subcat"])[["hi"]].stack().reset_index()
df.columns = ["cat", "subcat", "a", "value"]
df=df.drop('a',axis=1)
df['one']=1

In [ ]:
f = "fig2_polar"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")